This coloab notebook is used to retrain an existing model that is used for text recognition to use new custom data.  We then export the model for use in our application

In [ ]:
#NOTE*** This will fail in colab when running in colab with python verson 3.10
#this is a known bug but has not been fixed
!pip install -q tflite-model-maker

In [ ]:
import numpy as np
import os

from tflite_model_maker import configs
from tflite_model_maker import ExportFormat
from tflite_model_maker import model_spec
from tflite_model_maker import text_classifier
from tflite_model_maker.text_classifier import DataLoader

import tensorflow as tf
assert tf.__version__.startswith('2')
tf.get_logger().setLevel('ERROR')

In [ ]:
#pull a csv containing new training data given to us by the tutorial.  The training data is a list of comments with true/false values to
#whether or not the comment is spam
data_file = tf.keras.utils.get_file('comment-spam-extras.csv',
                                    origin='https://storage.googleapis.com/jmstore/TensorFlowJS/EdX/code/6.5/jm_blog_comments_extras.csv',
                                    extract=False)

In [ ]:
#configure the model and how it will use the training dataset
spec = model_spec.get('average_word_vec')
spec.num_words = 2000 #Choose the number of words that model will store as indications of spam (words in the vocabulary file)
spec.seq_len=20 #The number of tokens that the model will accept as an input
spec.wordvec_dim = 7 #The number of word vector dimentions to use to classify words

In [ ]:
#import the training data from the csv instantiated above
data = DataLoader.from_csv(file_path=data_file,
                           text_column='commenttext',
                           label_column='spam',
                           model_spec=spec,
                           deliteter=',',
                           shuffle=True,
                           is_training=True)


train_data, test_data = data.split(0.2)

In [ ]:
#train the model
model = text_classifier.create(train_data, model_spec=spec, epochs=50)

In [ ]:
#export the model for use in other places.  The first argument will specify the
#location of where to export the model files.  The second parameter specifies that
#model will use the TSJS format and will also export a label and vocabulary file
model.export(export_dir='/tmp/js_export',
             export_format=[ExportFormat.TFJS, ExportFormat.LABEL,
                            ExportFormat.VOCAB])

!zip -r /tmp/js_export/ModelFiles.zip /tmp/js_export/